https://colab.research.google.com/drive/1McyZC6yB1mbUqfpUQXccvTTReSwAm04e

In [0]:
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.recurrent import SimpleRNN
from keras.layers.embeddings import Embedding
from keras.layers import LSTM
import numpy as np
from keras.utils import to_categorical

Using TensorFlow backend.


In [0]:
#define documents
docs = ['this is','is an']
# define class labels
labels = ['an','example']


from collections import Counter
counts = Counter()
for i,review in enumerate(docs+labels):
  counts.update(review.split())
words = sorted(counts, key=counts.get, reverse=True)
vocab_size=len(words)
word_to_int = {word: i for i, word in enumerate(words, 1)}

encoded_docs = []
for doc in docs:
  encoded_docs.append([word_to_int[word] for word in doc.split()])
encoded_labels = []
for label in labels:
  encoded_labels.append([word_to_int[word] for word in label.split()])

In [0]:
# pad documents to a max length of 2 words
max_length = 2
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='pre')
print(padded_docs)

[[3 1]
 [1 2]]


In [0]:
# processing the output dataset
one_hot_encoded_labels = to_categorical(encoded_labels, num_classes=5)
print(one_hot_encoded_labels)

[[0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1.]]


In [0]:
# define the model
embed_length=1
max_length=2
model = Sequential()
model.add(LSTM(1,activation='tanh', return_sequences=False,recurrent_initializer='Zeros',recurrent_activation='sigmoid',
               input_shape=(max_length,embed_length),unroll=True))
model.add(Dense(5, activation='softmax'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 1)                 12        
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 10        
Total params: 22
Trainable params: 22
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
model.fit(padded_docs.reshape(2,2,1),np.array(one_hot_encoded_labels),epochs=500)

In [0]:
model.weights

[<tf.Variable 'lstm_2/kernel:0' shape=(1, 4) dtype=float32_ref>,
 <tf.Variable 'lstm_2/recurrent_kernel:0' shape=(1, 4) dtype=float32_ref>,
 <tf.Variable 'lstm_2/bias:0' shape=(4,) dtype=float32_ref>,
 <tf.Variable 'dense_2/kernel:0' shape=(1, 5) dtype=float32_ref>,
 <tf.Variable 'dense_2/bias:0' shape=(5,) dtype=float32_ref>]

In [0]:
model.get_weights()

[array([[ 0.01893798, -0.01265655,  0.45142433, -0.2514151 ]],
       dtype=float32),
 array([[0.19410396, 0.72569233, 0.79784596, 0.54982764]], dtype=float32),
 array([0.31918937, 1.5844553 , 0.15022673, 0.31409034], dtype=float32),
 array([[-0.38649145, -0.13294883,  0.969494  , -0.8276547 , -0.32456025]],
       dtype=float32),
 array([-0.44486812, -0.4477383 ,  0.42624277, -0.4402125 ,  0.46074733],
       dtype=float32)]

In [0]:
padded_docs[0]

array([3, 1], dtype=int32)

In [0]:
model.predict(padded_docs[0].reshape(1,2,1))

array([[0.10610605, 0.11620068, 0.41860935, 0.09055618, 0.26852772]],
      dtype=float32)

In [0]:
model.get_weights()[0]

array([[ 0.01893798, -0.01265655,  0.45142433, -0.2514151 ]],
      dtype=float32)

In [0]:
input_t0 = 3
cell_state0 = 0
forget0 = input_t0*model.get_weights()[0][0][1] + model.get_weights()[2][1]
forget1 = 1/(1+np.exp(-(forget0)))

In [0]:
cell_state1 = forget1 * cell_state0

In [0]:
input_t0_1 = input_t0*model.get_weights()[0][0][0] + model.get_weights()[2][0]
input_t0_2 = 1/(1+np.exp(-(input_t0_1)))

In [0]:
input_t0_cell1 = input_t0*model.get_weights()[0][0][2] + model.get_weights()[2][2]
input_t0_cell2 = np.tanh(input_t0_cell1)

In [0]:
input_t0_cell3 = input_t0_cell2*input_t0_2

In [0]:
input_t0_cell4 = input_t0_cell3 + cell_state1

In [0]:
output_t0_1 = input_t0*model.get_weights()[0][0][3] + model.get_weights()[2][3]
output_t0_2 = 1/(1+np.exp(-output_t0_1))

In [0]:
hidden_layer_1 = np.tanh(input_t0_cell4)*output_t0_2

In [0]:
input_t1 = 1
cell_state1 = input_t0_cell4

In [0]:
forget21 = hidden_layer_1*model.get_weights()[1][0][1] + model.get_weights()[2][1] + input_t1*model.get_weights()[0][0][1]
forget_22 = 1/(1+np.exp(-(forget21)))

In [0]:
cell_state2 = cell_state1 * forget_22

In [0]:
input_t1_1 = input_t1*model.get_weights()[0][0][0] + model.get_weights()[2][0] + hidden_layer_1*model.get_weights()[1][0][0]
input_t1_2 = 1/(1+np.exp(-(input_t1_1)))

In [0]:
input_t1_cell1 = input_t1*model.get_weights()[0][0][2] + model.get_weights()[2][2]+ hidden_layer_1*model.get_weights()[1][0][2]
input_t1_cell2 = np.tanh(input_t1_cell1)

In [0]:
input_t1_cell3 = input_t1_cell2*input_t1_2

In [0]:
input_t1_cell4 = input_t1_cell3 + cell_state2

In [0]:
output_t1_1 = input_t1*model.get_weights()[0][0][3] + model.get_weights()[2][3]+ hidden_layer_1*model.get_weights()[1][0][3]
output_t1_2 = 1/(1+np.exp(-output_t1_1))

In [0]:
hidden_layer_2 = np.tanh(input_t1_cell4)*output_t1_2

In [0]:
final_output = hidden_layer_2 * model.get_weights()[3][0] + model.get_weights()[4]

In [0]:
np.exp(final_output)/np.sum(np.exp(final_output))

array([0.10610606, 0.11620067, 0.41860938, 0.09055619, 0.26852775],
      dtype=float32)